# benchmarking

un Índice de Modelo Recursivo (RMI) y un B-Tree Ambos están diseñados para una cosa principal: encontrar datos rápidamente en un conjunto grande y ordenado. Pero, ¿cuál es mejor y bajo qué circunstancias?

Aquí es donde entra el benchmarking. Imagina que tienes dos autos deportivos y quieres saber cuál es más rápido, cuánto combustible gasta o cuánto espacio tiene para equipaje. El benchmarking es eso, pero para algoritmos y estructuras de datos. Nos permite medir y comparar su rendimiento de manera justa y objetiva.

En este cuaderno, aprenderemos a:

    Entender qué es el benchmarking y por qué es vital.
    Configurar un entorno para medir tiempos de construcción y búsqueda.
    Ejecutar pruebas con diferentes tamaños de datos.
    Recopilar los resultados para un análisis posterior.

### Preparación: Importando Nuestras Herramientas

Para ejecutar nuestro benchmark, necesitamos importar las clases y funciones que hemos construido en cuadernos anteriores:

    Nuestra implementación del RMI (rmi_model).
    Nuestra implementación del B-Tree (btree).
    Nuestras utilidades (utils) para generar datos y medir tiempo.

In [ ]:
import numpy as np
import time
import sys # Para medir el uso de memoria (aproximado)

# Importamos nuestras implementaciones
# Asegúrate de que tu ruta de Python esté configurada para encontrar el directorio 'src'
# Si estás ejecutando esto directamente en el directorio 'notebook', puede que necesites añadir esto:
import os
import sys
if os.getcwd().endswith('notebook'):
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../src')))

from rmi.rmi_model import RMI, train_linear_model, predict_linear_model # Importa la clase RMI y las funciones auxiliares
from btree.btree import BTree, BTreeNode # Importa la clase BTree y BTreeNode
from rmi.utils import generate_sorted_data, measure_time # Importa nuestras utilidades

print("Librerías y módulos importados correctamente.")

Métricas Clave del Benchmarking

Para comparar el RMI y el B-Tree, nos centraremos en tres métricas principales:

    Tiempo de Construcción del Índice: ¿Cuánto tiempo tarda cada estructura en organizarse para poder buscar datos? Para el RMI, es el tiempo de entrenamiento de sus modelos. Para el B-Tree, es el tiempo de insertar todos los elementos.
    Tiempo de Búsqueda de un Elemento: Una vez que el índice está construido, ¿cuánto tarda en encontrar un valor específico? Mediremos el tiempo promedio de muchas búsquedas para obtener una métrica representativa.
    Espacio Ocupado (Memoria): ¿Cuánta memoria RAM consume cada índice para almacenar su estructura? Esto es crucial para sistemas con recursos limitados.

Configuración de los Parámetros del Benchmark

Definiremos los tamaños de los datasets con los que queremos probar, el número de búsquedas a realizar para promediar los tiempos, y los parámetros específicos de cada estructura (como el orden del B-Tree o el número de modelos del Nivel 1 del RMI).

In [ ]:
# --- Parámetros del Benchmark ---

# Diferentes tamaños de datasets para probar
# Esto nos permitirá ver cómo escalan las estructuras con más datos.
dataset_sizes = [10_000, 100_000, 500_000, 1_000_000, 5_000_000, 10_000_000] # ¡Cuidado con el tiempo en los más grandes!

# Número de búsquedas a realizar para cada dataset, para obtener un promedio confiable
num_searches = 10_000

# Parámetros del B-Tree
B_TREE_ORDER = 100 # Un orden alto para simular nodos grandes, como bloques de disco

# Parámetros del RMI
# Este valor es crucial para el RMI. Más modelos = más precisión (y más memoria)
RMI_LEVEL1_MODELS = 1000 
# Buffer de la búsqueda local en RMI (qué tan amplio busca alrededor de la predicción)
RMI_SEARCH_WINDOW_BUFFER = 100 

print(f"Parámetros del B-Tree: Orden = {B_TREE_ORDER}")
print(f"Parámetros del RMI: Modelos Nivel 1 = {RMI_LEVEL1_MODELS}, Ventana de búsqueda = {RMI_SEARCH_WINDOW_BUFFER}")
print(f"Número de búsquedas por test: {num_searches}")

Estructura para Almacenar los Resultados

Vamos a almacenar los resultados en un diccionario o una lista de diccionarios para poder analizar y visualizar fácilmente más tarde.

In [ ]:
# --- Almacenamiento de Resultados ---
results = [] # Lista para guardar los resultados de cada tamaño de dataset

Ejecutando el Benchmark Paso a Paso

Ahora, crearemos un bucle que iterará sobre cada tamaño de dataset, realizará las operaciones y medirá el tiempo.

In [ ]:
print("\n--- Iniciando el Proceso de Benchmarking ---")

for size in dataset_sizes:
    print(f"\n--- Probando con un dataset de tamaño: {size:,} ---")
    
    # 1. Generar Datos
    # Usamos nuestra utilidad para asegurar que los datos estén ordenados
    data, gen_time = measure_time(generate_sorted_data, size)
    print(f"Datos generados en {gen_time:.4f} segundos.")

    # 2. Preparar valores de búsqueda aleatorios (que existan en el dataset)
    # Esto asegura que estamos buscando valores que realmente se pueden encontrar.
    search_keys_indices = np.random.randint(0, size, num_searches)
    search_keys = data[search_keys_indices]
    
    # --- Benchmarking del RMI ---
    print("\n  >> RMI:")
    
    # Construcción del RMI
    # Recordamos los parámetros que le pasamos al constructor de RMI
    rmi_instance, rmi_build_time = measure_time(RMI, data, RMI_LEVEL1_MODELS, RMI_SEARCH_WINDOW_BUFFER)
    print(f"    Tiempo de construcción del RMI: {rmi_build_time:.6f} segundos.")

    # Búsqueda en el RMI
    total_rmi_search_time = 0
    for key in search_keys:
        _, search_time = measure_time(rmi_instance.search, key)
        total_rmi_search_time += search_time
    avg_rmi_search_time = total_rmi_search_time / num_searches
    print(f"    Tiempo promedio de búsqueda del RMI ({num_searches} búsquedas): {avg_rmi_search_time:.9f} segundos.")

    # Medida de memoria del RMI (aproximada)
    # Esto es complejo y sys.getsizeof no es preciso para estructuras anidadas.
    # Para una medida más precisa se necesitaría algo como `pympler.asizeof` o análisis profundo.
    # Aquí, una estimación muy básica del objeto principal, no de todos los componentes internos.
    rmi_memory = sys.getsizeof(rmi_instance) + sys.getsizeof(rmi_instance.level0_model) + \
                 sum(sys.getsizeof(m) for m in rmi_instance.level1_models if m is not None)
    print(f"    Uso de memoria del RMI (aprox.): {rmi_memory / (1024*1024):.4f} MB.")


    # --- Benchmarking del B-Tree ---
    print("\n  >> B-Tree:")

    # Construcción del B-Tree
    # Reiniciamos el B-Tree para cada prueba de tamaño de datos
    b_tree_instance = BTree(B_TREE_ORDER)
    
    # Insertar todas las claves en el B-Tree
    b_tree_build_start = time.perf_counter()
    for key in data: # Insertamos todos los datos originales
        b_tree_instance.insert(key)
    b_tree_build_time = time.perf_counter() - b_tree_build_start
    print(f"    Tiempo de construcción del B-Tree: {b_tree_build_time:.6f} segundos.")

    # Búsqueda en el B-Tree
    total_btree_search_time = 0
    for key in search_keys:
        _, search_time = measure_time(b_tree_instance.search, key)
        total_btree_search_time += search_time
    avg_btree_search_time = total_btree_search_time / num_searches
    print(f"    Tiempo promedio de búsqueda del B-Tree ({num_searches} búsquedas): {avg_btree_search_time:.9f} segundos.")

    # Medida de memoria del B-Tree (aproximada y más compleja de calcular con precisión)
    # Calcular la memoria exacta de un B-Tree Python con muchos nodos es difícil sin herramientas de profiling.
    # sys.getsizeof solo da el tamaño del objeto "BTree" o "BTreeNode", no de todo el árbol recursivamente.
    # Para una estimación básica de un nodo, podemos ver:
    # b_tree_memory_per_node = sys.getsizeof(BTreeNode(B_TREE_ORDER, True)) # Tamaño de un nodo vacío
    # b_tree_approx_nodes = size / (B_TREE_ORDER -1) # Estimación del número de nodos (muy burda)
    # b_tree_total_memory_approx = b_tree_approx_nodes * b_tree_memory_per_node # Muy impreciso
    # Para esta demo, lo dejaremos como un placeholder.
    # Una medida más precisa requeriría `pympler.asizeof` o un análisis recursivo.
    
    # Placeholder para memoria del B-Tree - Nota: La memoria real de un B-Tree es muy compleja de calcular
    # con sys.getsizeof debido a la estructura de nodos y enlaces. Esto es solo una aproximación.
    # En un escenario real, usaríamos herramientas de profiling de memoria.
    btree_memory = sys.getsizeof(b_tree_instance) # Tamaño del objeto BTree
    # Puedes añadir aquí una lógica para recorrer nodos y sumar sus tamaños, pero es complejo.
    print(f"    Uso de memoria del B-Tree (objeto principal, aprox.): {btree_memory / (1024*1024):.4f} MB.")


    # --- Guardar Resultados ---
    results.append({
        'dataset_size': size,
        'rmi_build_time': rmi_build_time,
        'rmi_avg_search_time': avg_rmi_search_time,
        'rmi_memory_mb': rmi_memory / (1024*1024),
        'btree_build_time': b_tree_build_time,
        'btree_avg_search_time': avg_btree_search_time,
        'btree_memory_mb': btree_memory / (1024*1024) # Placeholder
    })

print("\n--- Benchmarking Completado ---")

# Mostrar un resumen de los resultados
print("\n--- Resumen de Resultados ---")
for r in results:
    print(f"\nTamaño: {r['dataset_size']:,}")
    print(f"  RMI  - Construcción: {r['rmi_build_time']:.6f} s, Búsqueda: {r['rmi_avg_search_time']:.9f} s, Memoria (aprox): {r['rmi_memory_mb']:.4f} MB")
    print(f"  B-Tree - Construcción: {r['btree_build_time']:.6f} s, Búsqueda: {r['btree_avg_search_time']:.9f} s, Memoria (aprox): {r['btree_memory_mb']:.4f} MB")

Análisis Preliminar de los Resultados

Al observar los resultados impresos, incluso antes de graficarlos, podemos empezar a ver patrones:

    Tiempo de Construcción: Generalmente, construir un B-Tree implica insertar cada elemento uno por uno, lo que puede ser más lento para grandes datasets. El RMI, por otro lado, entrena modelos en una pasada, lo que a menudo es más rápido.
    Tiempo de Búsqueda: Aquí es donde el RMI busca destacar. Su objetivo es acercarse a la velocidad de una búsqueda binaria (o incluso superarla) en promedio, mientras que el B-Tree tiene una complejidad logarítmica garantizada, pero con más "saltos" entre nodos.
    Uso de Memoria: Este es un punto clave. El RMI almacena solo los parámetros de sus modelos (unos pocos números por modelo), lo que tiende a ser muy compacto. El B-Tree almacena cada clave y puntero en sus nodos, lo que puede consumir significativamente más memoria para datasets muy grandes.

Los números exactos variarán en cada ejecución y máquina, pero las tendencias generales deberían ser evidentes.